In [ ]:
# --------------------
# 2) 교차 검증 후 코드 (K-Fold를 통한 일반화 성능 평가)
# --------------------

print("\n\n#####################################################")
print("## 2. 교차 검증 후 (일반화 성능 평가)")
print("#####################################################")

# 종속변수/독립변수를 Scikit-learn 형식으로 준비
y = df_encoded['disease']
X = df_encoded.drop('disease', axis=1)

# 1단계: Stratified K-Fold 설정 (K=5)
# 분류 문제이므로 클래스 비율을 유지하는 StratifiedKFold 사용
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 2단계: Scikit-learn 로지스틱 회귀 모델 정의
model_cv = LogisticRegression(random_state=42)

# 3단계: cross_val_score 함수로 교차 검증 실행
cv_scores = cross_val_score(
    model_cv,       # Scikit-learn 모델
    X, y,           # 전체 데이터
    cv=skf,         # K-Fold 설정
    scoring='accuracy', # 평가 지표
    n_jobs=-1
)

# 4단계: 최종 결과 출력
mean_cv_accuracy = cv_scores.mean()
std_cv_accuracy = cv_scores.std()

print("\n--- K-Fold 교차 검증 (5-Fold) 결과 ---")
print(f"각 Fold의 정확도: {cv_scores}")
print(f"교차 검증 평균 정확도: {mean_cv_accuracy:.4f}")
print(f"정확도 표준 편차: {std_cv_accuracy:.4f}")

# 해석: 이 평균 정확도 값은 모델이 새로운 데이터에 대해 기대할 수 있는 성능을 객관적으로 나타냅니다.

In [ ]:
#--------------------------------------------
# 머신러닝 학습 및 평가 (cross_val_score를 사용한 K-Fold 교차 검증 - NumPy 대신 statistics 사용)
#--------------------------------------------
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error, make_scorer
# import numpy as np  # NumPy를 사용하지 않음
import statistics # 표준 라이브러리 statistics 모듈 사용
import lightgbm as lgb
import warnings

# LightGBM에서 verbose=-1 설정 시 경고가 발생할 수 있어 무시합니다.
warnings.filterwarnings('ignore', category=UserWarning)


# K-Fold 설정 (예: 5-Fold)
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# LightGBM 모델 생성
model = lgb.LGBMRegressor(random_state=0, verbose=-1)

# cross_val_score를 사용하여 각 지표의 평균 점수를 계산합니다.
# scikit-learn은 점수를 최대화하는 방식으로 동작하므로, 
# MSE와 MAE에는 'neg_' 접두사를 붙여 음수로 변환한 후, 결과를 다시 양수로 바꿉니다.

# 1. MSE (Negated Mean Squared Error)
# cross_val_score는 결과를 numpy 배열로 반환하지만, 이후에 리스트로 변환하여 처리합니다.
mse_scores_neg = cross_val_score(
    model, 
    train, 
    target, 
    cv=kf, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1
).tolist() # NumPy 배열을 Python 리스트로 변환

# 2. RMSE (MSE를 루트 씌움)
# 각 MSE 점수에 대해 루트를 씌우고, 결과를 리스트로 저장합니다.
rmse_scores = [val**0.5 for val in [-score for score in mse_scores_neg]]

# 3. MAE (Negated Mean Absolute Error)
mae_scores_neg = cross_val_score(
    model, 
    train, 
    target, 
    cv=kf, 
    scoring='neg_mean_absolute_error', 
    n_jobs=-1
).tolist() # NumPy 배열을 Python 리스트로 변환

# 4. R2 (R-squared)
r2_scores = cross_val_score(
    model, 
    train, 
    target, 
    cv=kf, 
    scoring='r2', 
    n_jobs=-1
).tolist() # NumPy 배열을 Python 리스트로 변환

# 최종 평균 성능 지표 출력
print("="*50)
print(f"Final Average Cross-Validation Results ({n_splits} Folds) using cross_val_score:")
print("="*50)

# MSE (음수 결과를 다시 양수로 변환하고 statistics 사용)
mse_scores = [-score for score in mse_scores_neg]
avg_mse = statistics.mean(mse_scores)
std_mse = statistics.stdev(mse_scores)
print(f'Average MSE: {avg_mse:.4f} (Std: {std_mse:.4f})')

# RMSE
avg_rmse = statistics.mean(rmse_scores)
std_rmse = statistics.stdev(rmse_scores)
print(f'Average RMSE: {avg_rmse:.4f} (Std: {std_rmse:.4f})')

# MAE (음수 결과를 다시 양수로 변환하고 statistics 사용)
mae_scores = [-score for score in mae_scores_neg]
avg_mae = statistics.mean(mae_scores)
std_mae = statistics.stdev(mae_scores)
print(f'Average MAE: {avg_mae:.4f} (Std: {std_mae:.4f})')

# R2
avg_r2 = statistics.mean(r2_scores)
std_r2 = statistics.stdev(r2_scores)
print(f'Average R2: {avg_r2:.4f} (Std: {std_r2:.4f})')
print("="*50)